In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
import keras
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras import backend as K
from keras.models import Sequential,Input,Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd
import random as rn
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
from scipy import linalg
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.io import loadmat
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import time
%matplotlib inline
import itertools


# Loading features 
import json
with open('feature_data.json', 'r') as f:
    features = json.load(f)

features_ar = np.array(features)

# Normalising features
# scaler = StandardScaler()
# scaler.fit(features_ar)
# features_ar = scaler.transform(features_ar)

train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
camIds = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()
filelists = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()

train_idxss = np.subtract(train_idxs, 1)
gallery_idxss = np.subtract(gallery_idxs, 1)
query_idxss = np.subtract(query_idxs, 1)

camIds_train = camIds[train_idxss]
filelists_train = filelists[train_idxss]
labels_train = labels[train_idxss]
features_train = features_ar[train_idxss]

camIds_gal = camIds[gallery_idxss]
filelists_gal = filelists[gallery_idxss]
labels_gal = labels[gallery_idxss]
features_gal = features_ar[gallery_idxss]

camIds_query = camIds[query_idxss]
filelists_query = filelists[query_idxss]
labels_query = labels[query_idxss]
features_query = features_ar[query_idxss]




Using TensorFlow backend.


In [ ]:
# Stratified train test split for training the network
x_train, x_test, y_train, y_test, z_train, z_test  = train_test_split(features_train, labels_train, camIds_train, test_size=0.33, random_state=13, stratify=labels_train)

In [ ]:
# Produces a list of lists corresponding to indexes of elements in each class
def dig_index(label):
    length = len(np.unique(label))
    setClass = list(set(label))
    class_ind = []
    for i in setClass:
        tempInd = []
        for m in range(0,len(label)):
            if i == label[m]:
                tempInd.append(m)
        class_ind.append(tempInd)
    return class_ind


In [ ]:
# Creates a matrix of copies of input feature to a desired height, for pairing
def big_query(featureList, ind, length):
    query_copy = np.empty([length,2048])
    for i in range(0, length):
        query_copy[i] = featureList[ind]
    return query_copy

def big_query_train(featureList, ind, length):
    query_copy = np.empty([length,64,32])
    for i in range(0, length):
        query_copy[i] = featureList[ind]
    return query_copy


In [5]:
# Generates all possible positive pairs
def pos_pair(label):
    pair_ind = []
    digit_indices  = np.asarray(dig_index(label))
    for i in digit_indices:
        tempInd = list(itertools.permutations(i, 2))
        pair_ind.append(tempInd)
    return pair_ind

In [6]:
# # Generates hardest negative pairs for each point by finding closest 10 with different classes from it
# k=40
# inds_wp =[]
# n_query = len(y_train)
# n_gal = len(y_train)
# for i in range(0, n_query):
#     query_l = y_train[i]
#     print('Query images used to create negatives = ', i, end="\r")
#     big_q = big_query(x_train, i, len(x_train))
#     dist = np.linalg.norm(big_q - x_train, axis=1)      
#     ids = np.copy(z_train)
#     lbs = np.copy(y_train)
#     inds = list(range(0, n_gal))
#     neigh = []
#     while len(neigh)<k:
#         rem = False
#         mind = np.argmin(dist)
#         ind = inds[mind]
#         if ((ids[mind] == z_train[i]) and (lbs[mind] == y_train[i])) == True:            
#             rem = True
#         if rem == False:
#             neigh = np.append(neigh, [mind], axis=0)
#         dist = np.delete(dist, mind, axis=0)
#         inds = np.delete(inds, mind, axis=0)
#         ids = np.delete(ids, mind, axis=0)
#         lbs = np.delete(lbs, mind, axis=0)

#     count = 0
#     z = 0
#     while count<10:
#         if(y_train[int(neigh[z])]!=y_train[i]):
#             ind_wp = [i, neigh[z]]
#             inds_wp.append(ind_wp)
#             count=count+1
#         z=z+1    

In [7]:
# k=40
# inds_wpte =[]
# n_query = len(y_test)
# n_gal = len(y_test)
# for i in range(0, n_query):
#     query_l = y_test[i]
#     print('i = ', i, end="\r")
#     dist = np.linalg.norm(big_query(x_test, i, len(x_test)) - x_test, axis=1)        
#     ids = np.copy(z_test)
#     lbs = np.copy(y_test)
#     inds = list(range(0, n_gal))
#     neigh = []
#     while len(neigh)<k:
#         rem = False
#         mind = np.argmin(dist)
#         ind = inds[mind]
#         if ((ids[mind] == z_test[i]) and (lbs[mind] == y_test[i])) == True:            
#             rem = True
#         if rem == False:
#             neigh = np.append(neigh, [mind], axis=0)
#         dist = np.delete(dist, mind, axis=0)
#         inds = np.delete(inds, mind, axis=0)
#         ids = np.delete(ids, mind, axis=0)
#         lbs = np.delete(lbs, mind, axis=0)

#     count = 0
#     z = 0
#     while count<10:
#         if(y_test[int(neigh[z])]!=y_test[i]):
#             ind_wp = [i, neigh[z]]
#             inds_wpte.append(ind_wp)
#             count=count+1
#         z=z+1


In [8]:
def prune_pairs(posInd, cam):
    for i in posInd:
        if cam[i[0]] == cam[i[1]]:
            posInd = np.delete(posInd, i, 0)
    return posInd
           

In [ ]:
# pos_indx_te = pos_pair(y_test)
# flat_list_te = [item for sublist in pos_indx_te for item in sublist]
# flat_list_pos_te = prune_pairs(flat_list_te, z_test)
# label_pairs_te = np.zeros([len(inds_wpte)])
# label_sames_te = np.ones([len(flat_list_pos_te)])
# label_pairs_te = np.append(label_pairs_te, label_sames_te)
# flat_arr_te = [list(elem) for elem in flat_list_pos_te]
# full_pairs_te = np.vstack((inds_wpte, flat_arr_te))
# print(full_pairs_te.shape)

# df = pd.DataFrame(full_pairs_te)
# df.to_csv("full_pairs_pos_te.csv", header=None, index=None)
# df = pd.DataFrame(label_pairs_te)
# df.to_csv("labels_pairs_pos_te.csv", header=None, index=None)

In [ ]:
# pos_indx = pos_pair(y_train)
# flat_list = [item for sublist in pos_indx for item in sublist]
# flat_list_pos = prune_pairs(flat_list, z_train)
# label_pairs = np.zeros([len(inds_wp)])
# label_sames = np.ones([len(flat_list_pos)])

# label_pairs = np.append(label_pairs, label_sames)
# flat_arr = [list(elem) for elem in flat_list_pos]
# full_pairs = np.vstack((inds_wp, flat_arr))
# print(full_pairs.shape)

# df = pd.DataFrame(full_pairs)
# df.to_csv("full_pairs_pos.csv", header=None, index=None)
# df = pd.DataFrame(label_pairs)
# df.to_csv("labels_pairs_pos.csv", header=None, index=None)

In [ ]:
df = pd.read_csv("full_pairs_pos.csv", header=None, index_col=None)
full_pairs = np.array([df.values])
df = pd.read_csv("full_pairs_pos_te.csv", header=None, index_col=None)
full_pairs_te = np.array([df.values])
df = pd.read_csv("labels_pairs_pos.csv", header=None, index_col=None)
label_pairs = np.array([df.values])
df = pd.read_csv("labels_pairs_pos_te.csv", header=None, index_col=None)
label_pairs_te = np.array([df.values])

In [ ]:
full_pairs_te = full_pairs_te.astype(np.int64)
full_pairs = full_pairs.astype(np.int64)
label_pairs = label_pairs.astype(np.int64)
label_pairs_te = label_pairs_te.astype(np.int64)
full_pairs = full_pairs[0]
full_pairs_te = full_pairs_te[0]
label_pairs = label_pairs[:][0]
label_pairs_te = label_pairs_te[:][0]
label_pairs = label_pairs.flatten()
label_pairs_te = label_pairs_te.flatten()

In [ ]:
dig_indx  = np.asarray(dig_index(y_test))

In [ ]:
# x_train, x_test, y_train, y_test, z_train, z_test  = train_test_split(features_train.reshape(-1,64,32), labels_train, camIds_train, test_size=0.33, random_state=13, stratify=labels_train)
x_train, x_test, y_train, y_test, z_train, z_test  = train_test_split(features_train.reshape(-1,2048,1), labels_train, camIds_train, test_size=0.33, random_state=13, stratify=labels_train)


In [ ]:
'''Trains a Siamese MLP on pairs of digits from the MNIST dataset.

It follows Hadsell-et-al.'06 [1] by computing the Euclidean distance on the
output of the shared network and by optimizing the contrastive loss (see paper
for mode details).

# References

- Dimensionality Reduction by Learning an Invariant Mapping
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf

'''

num_classes = 767
epochs = 10


def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 0.25
    sqaure_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * sqaure_pred + (1 - y_true) * margin_square)


def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    print(n)
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)


def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
#     x = Conv2D(32, kernel_size=(2, 2),activation='linear',input_shape=(2048,1,1),padding='same')(input)
#     x = Dropout(0.25)(x)
#     x = Conv2D(64, (2, 2), activation='linear',padding='same')(x)
#     x = Dropout(0.5)()
    x = Dense(128, activation='relu')(input)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)


def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.125
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.125, y_true.dtype)))


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# x_train = x_train.reshape(-1,2048,1) 
# x_test = x_test.reshape(-1,2048,1) 
input_shape = x_train.shape[1:]
# input_shape = (2048,1,1)


In [ ]:
# network definition
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
model.fit([x_train[full_pairs[:, 0]], x_train[full_pairs[:, 1]]], label_pairs,
          batch_size=64,
          epochs=epochs,
          validation_data=([x_test[full_pairs_te[:, 0]], x_test[full_pairs_te[:, 1]]], label_pairs_te))
# model.load_weights('weights.hdf5', by_name=False)

# compute final accuracy on training and test sets
y_pred = model.predict([x_train[full_pairs[:, 0]], x_train[full_pairs[:, 1]]])
tr_acc = compute_accuracy(label_pairs, y_pred)
y_pred = model.predict([x_test[full_pairs_te[:, 0]], x_test[full_pairs_te[:, 1]]])
te_acc = compute_accuracy(label_pairs_te, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

Train on 52606 samples, validate on 25436 samples
Epoch 1/10
52606/52606 [==============================] - 87s 2ms/step - loss: 0.0293 - accuracy: 0.8667 - val_loss: 0.0070 - val_accuracy: 0.8758
Epoch 2/10
52606/52606 [==============================] - 81s 2ms/step - loss: 0.0061 - accuracy: 0.8775 - val_loss: 0.0066 - val_accuracy: 0.8696
Epoch 3/10
52606/52606 [==============================] - 81s 2ms/step - loss: 0.0054 - accuracy: 0.8916 - val_loss: 0.0065 - val_accuracy: 0.8893
Epoch 4/10
52606/52606 [==============================] - 81s 2ms/step - loss: 0.0050 - accuracy: 0.8967 - val_loss: 0.0069 - val_accuracy: 0.8899
Epoch 5/10
52606/52606 [==============================] - 81s 2ms/step - loss: 0.0050 - accuracy: 0.8987 - val_loss: 0.0069 - val_accuracy: 0.8948
Epoch 6/10
52606/52606 [==============================] - 81s 2ms/step - loss: 0.0046 - accuracy: 0.9054 - val_loss: 0.0066 - val_accuracy: 0.8981ccuracy: 0. - ETA: 37s - l - ETA: 22s - loss: 0 - ETA: 0s - loss: 0.0

In [ ]:
# accuracy = model.history['acc']
# val_accuracy = model.history['val_acc']
# loss = model.history['loss']
# val_loss = model.history['val_loss']
# epochs = range(len(accuracy))
# plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
# plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
# plt.title('Training and validation accuracy')
# plt.legend()
# plt.figure()
# plt.plot(epochs, loss, 'bo', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()
# plt.show()

In [ ]:
def label_instances(labels_g, labels_q, Ids_g, Ids_q):
#     construct a empty set of length n_query
#     go through allquery labels
#     see how many query labels of the same type but different camera exist in the gal set
    n_q = len(Ids_q)
    n_g = len(Ids_g)
    instances = [0]*n_q
    for i in range(0, n_q):
        for m in range(0, n_g):
            if (labels_g[m] == labels_q[i]) and not (Ids_g[m] == Ids_q[i]):
                instances[i] = instances[i] + 1
    return(instances)

In [ ]:
def rankKeval(labels_neigh, label_query):
    contained = False
    for i in range (0, len(labels_neigh)):
        if (labels_neigh[i] == label_query):
            contained = True
    return(contained)

def scores_query(ranklist, query_l, instances, k):
    acc = 0
    TP = 0
    FP = 0
    prec = 0
    rec = 0
    
    if rankKeval(ranklist, query_l) == True:
        acc = 1
        
    for i in range (0, k):
        if (ranklist[i] == query_l):
            TP = TP + 1
        else:
            FP = FP +1
    prec = TP/(TP + FP)
    rec = TP/instances
    
    return(acc, prec, rec)

def scores_k(acc_l, prec_l, rec_l, k, query_n):
    acc_avg = []
    prec_avg = []
    rec_avg = []
    acc_avg.append(sum(acc_l)/query_n)
    prec_avg.append(sum(prec_l)/query_n)
    rec_avg.append(sum(rec_l)/query_n)
    return(acc_avg, prec_avg, rec_avg)

def average(lst, lstlen):
    avg = 0
    for i in range(0, lstlen-1):
        avg = avg + lst[i][0]
    avg = avg/lstlen
    return(avg)

In [ ]:
def do_kNN_new(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, k, s_net, instances):
    n_gal = len(labels_gal)
    query_n = len(labels_query)
    print(query_n)
    acc = [0] * (query_n * k)
    prec = [0] * (query_n * k)
    rec = [0] * (query_n * k)
    AA = [0] * k
    AR = [0] * k 
    AP = [0] * k
    F1 = [0] * k
    mAP = [0] * k 
    for i in range (0, query_n):
        print('i = ', i, end="\r")

        query_l = labels_query[i]
        query_f = features_query[i]
        query_Id = camIds_query[i]
        
        
        dist = model.predict([big_query(features_query, i, len(features_gal)).reshape(-1,2048, 1), features_gal.reshape(-1,2048, 1)])
        dist = dist[:,0]
        
        ids = np.copy(camIds_gal)
        lbs = np.copy(labels_gal)
        inds = list(range(0, n_gal))
        neigh = []
        while len(neigh)<k:
            rem = False
            mind = np.argmin(dist)
            ind = inds[mind]
            if ((ids[mind] == camIds_query[i]) and (lbs[mind] == labels_query[i])):
                rem = True
            if rem == False:
                neigh = np.append(neigh, [lbs[mind]], axis=0)
            dist = np.delete(dist, mind, axis=0)
            inds = np.delete(inds, mind, axis=0)
            ids = np.delete(ids, mind, axis=0)
            lbs = np.delete(lbs, mind, axis=0)
        
        
        
        for z in range (0, k):
            ind = i+(z*query_n)
            acc[ind], prec[ind], rec[ind] = scores_query(neigh, query_l, instances[i], (k-z))

            neigh = np.delete(neigh, len(neigh)-1, 0)
    for i in range (0, k):
        LB = i*query_n
        UB = (i+1)*query_n
        AA[i], AP[i], AR[i] = scores_k(acc[LB:UB], prec[LB:UB], rec[LB:UB], i+1, query_n)
#         F1[i] = 2*(AP[i][0]*AR[i][0])/(AP[i][0]+AR[i][0])
    mAP[i] = 1
    return(AA, AR, AP, mAP, F1)

In [ ]:
instances = label_instances(labels_gal, labels_query, camIds_gal, camIds_query)


In [ ]:
AA, AR, AP, mAP, F1 = do_kNN_new(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, 10, 4, instances)

1400


In [ ]:
print("Results for up to rank 10")
print("AA: ", AA)
print("F1: ", F1)
print("AR: ",AR)
print("AP: ",AP)
print("mAP: ",mAP)

In [ ]:
# def transform_features(data):
#     x = data.reshape(-1, 2048, 1, 1)
#     get_3rd_layer_output = K.function([model.layers[0].input],
#                                       [model.layers[6].output])
#     feature_output = get_3rd_layer_output([x])[0]
#     test = feature_output.reshape(-1, 2048)
#     return test

In [ ]:
features_gal_tf = transform_features(features_gal[0:2000])
# print(features_gal_tf.shape)
features_gal_tf = np.vstack((features_gal_tf, transform_features(features_gal[2000:4000])))
# print(features_gal_tf.shape)
features_gal_tf = np.vstack((features_gal_tf, transform_features(features_gal[4000:])))
# print(features_gal_tf.shape)
features_query_tf = transform_features(features_query)
# print(features_query_tf.shape)

In [ ]:
# model.save_weights('weights.hdf5')